In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize
from prettytable import PrettyTable
from IPython.display import display

In [ ]:
tickers = ["BHP.AX", "GOOGL", "AMZN", "WMT", "XOM", "BRK-A", "UNH", "UPS", "AMT", "AAPL", "NEE"]

# Download historical data from Yahoo Finance
data = yf.download(tickers, start="2019-02-01", end="2021-12-30")

# Filter the "Adj Close" column for daily returns
returns = data["Adj Close"].pct_change()

# Define a dictionary with ticker and company names
companies = {
    "BHP.AX": "BHP Group Limited",
    "GOOGL": "Alphabet.com, Inc.",
    "AMZN": "Amazon.com, Inc.",
    "WMT": "Walmart Inc.",
    "XOM": "Exxon Mobil Corporation",
    "BRK-A": "Berkshire Hathaway Inc.",
    "UNH": "UnitedHealth Group Incorporated",
    "UPS": "United Parcel Service, Inc.",
    "AMT": "American Tower Corporation",
    "AAPL": "Apple Inc.",
    "NEE": "NextEra Energy, Inc."
}

In [ ]:
table = PrettyTable()
table.field_names = ["Ticker", "Company Name", "Market Cap", "Industry"]

In [ ]:
for ticker in returns.columns:
    company_name = companies.get(ticker, "N/A")
    market_cap = yf.Ticker(ticker).info.get("marketCap", "N/A")
    industry = yf.Ticker(ticker).info.get("industry", "N/A")
    table.add_row([ticker, company_name, market_cap, industry])

# Display the table
display(table)

In [ ]:
from scipy.stats import skew, kurtosis
table = PrettyTable()
table.field_names = ["Stock", "Max", "Min", "Mean", "Standard Deviation", "Cumulative Return", "Skewness", "Kurtosis"]

# Populate the table with data
for ticker in returns.columns:
    stock_name = companies.get(ticker, "N/A")
    max_return = returns[ticker].max()
    min_return = returns[ticker].min()
    mean_return = returns[ticker].mean()
    std_return = returns[ticker].std()
    cumulative_return = (1 + returns[ticker]).cumprod()[-1] - 1
    skewness = skew(returns[ticker].dropna())
    kurt = kurtosis(returns[ticker].dropna())
    
    table.add_row([stock_name[:20], f"{max_return:.4f}", f"{min_return:.4f}", f"{mean_return:.4f}", f"{std_return:.4f}",
                   f"{cumulative_return:.4f}", f"{skewness:.4f}", f"{kurt:.4f}"])

# Display the table
display(table)